In [22]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report

In [23]:
# Gerekli NLTK verilerini indir
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [24]:
# Veriyi okudum
df = pd.read_csv("spam_ham_dataset.csv")
df.head()

,Unnamed: 0,label,text,label_num
0,605,ham,Subject: enron methanol ; meter # : 988291\nth...,0
1,2349,ham,"Subject: hpl nom for january 9 , 2001\n( see a...",0
2,3624,ham,"Subject: neon retreat\nho ho ho , we ' re arou...",0
3,4685,spam,"Subject: photoshop , windows , office . cheap ...",1
4,2030,ham,Subject: re : indian springs\nthis deal is to ...,0


In [25]:
#  Label sayısal değilse dönüştürme
df['label_num'] = df['label'].map({'ham': 0, 'spam': 1})

In [26]:
# E-posta metinlerini temizlemek için bir fonksiyon yazıyoruz
def clean_text(text):
    text = text.lower()  # Küçük harfe çevir
    text = re.sub(r'\d+', '', text)  # Sayıları kaldır
    text = re.sub(r'[^\w\s]', '', text)  # Noktalama işaretlerini kaldır
    text = re.sub(r'\s+', ' ', text).strip()  # Fazla boşlukları düzelt
    text = " ".join([word for word in text.split() if word not in stop_words])  # Stopword'leri kaldır
    return text

# Bu fonksiyonu tüm e-posta metinlerine uyguluyoruz
df['clean_text'] = df['text'].apply(clean_text)
#Bu adımda metinler temizleniyor ve gereksiz kelimeler çıkarılıyor. Bu işlem modelin daha iyi öğrenmesini sağlar.

In [27]:
# TF-IDF (Term Frequency - Inverse Document Frequency) ile metinleri sayısal vektörlere dönüştürüyoruz
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df['clean_text'])  # Özellikler (bağımsız değişkenler)
y = df['label_num']  # Etiketler (bağımlı değişken)
# TF-IDF, kelimelerin önem derecesini belirleyerek metinleri sayısal verilere çevirir. Bu, KNN gibi algoritmaların metinler üzerinde çalışmasını sağlar.

In [28]:
# Veriyi eğitim ve test olarak ikiye ayırıyoruz
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
#Verilerin %80’i modelin eğitimi için, %20’si test için kullanılır.

In [29]:
# KNN modelini oluşturuyoruz ve cosine benzerliğine göre çalışmasını sağlıyoruz
knn = KNeighborsClassifier(n_neighbors=5, metric='cosine')
# Modeli eğitiyoruz
knn.fit(X_train, y_train)
#KNeighborsClassifier sınıfı ile KNN modeli kuruluyor. cosine metriği, metin verilerinde daha iyi sonuç verir.

KNeighborsClassifier(metric='cosine')

In [30]:
# Test verileri ile tahmin yapıyoruz
y_pred = knn.predict(X_test)
# Model performansını değerlendiriyoruz
print(classification_report(y_test, y_pred))
#Bu metrikler sayesinde modelin doğruluğu, hassasiyeti, hatırlama oranı ve F1 skorları ölçülür.

              precision    recall  f1-score   support

           0       0.96      0.99      0.97       742
           1       0.97      0.89      0.93       293

    accuracy                           0.96      1035
   macro avg       0.97      0.94      0.95      1035
weighted avg       0.96      0.96      0.96      1035



**Model Performans Değerlendirmesi**


**Performans Ölçütleri:**

Sınıf 0:

Kesinlik (Precision): 0.96
Duyarlılık (Recall): 0.99
F1-Score: 0.97
Destek (Support): 742

Sınıf 1:

Kesinlik (Precision): 0.97
Duyarlılık (Recall): 0.89
F1-Score: 0.93
Destek (Support): 293

**Genel Performans:**

Modelimizin genel doğruluğu %96 olarak hesaplanmıştır.

Doğruluk (Accuracy): Modelin genel doğruluğunun %96 , yani toplamda doğru tahminler yapıyor.

Makro ve Ağırlıklı Ortalama: Makro ortalamalar, her sınıfın eşit ağırlıkta değerlendirildiği, ağırlıklı ortalama ise sınıf desteklerine dayalı bir değerlendirmedir. Bu, modelin genel denge durumunu gösterir.

Bu, modelin toplamda doğru tahminler yaptığını gösteriyor. Ayrıca, makro ve ağırlıklı ortalamalar da şu şekildedir:

**Makro Ortalama:**

Kesinlik: 0.97
Duyarlılık: 0.94
F1-Score: 0.95
Ağırlıklı Ortalama:

Kesinlik: 0.96
Duyarlılık: 0.96
F1-Score: 0.96

**Sonuçların Değerlendirilmesi:**

Modelimizin Sınıf 0 için yüksek bir kesinlik ve duyarlılık oranı vardır. Bu, modelin bu sınıfı doğru bir şekilde tanımlamakta oldukça başarılı olduğunu gösteriyor. Ancak Sınıf 1 için duyarlılığın 0.89 olması, modelin bu sınıfı tanımlamada bazı zorluklar yaşadığını ortaya koyuyor. Bu durum, yanlış pozitif oranının daha yüksek olabileceği anlamına gelebilir.

**Geliştirme:**

Sınıf 1 için duyarlılığı artırmak amacıyla daha fazla veri toplayabilir veya veri artırma tekniklerini uygulayabiliriz.
    
Modelin parametrelerini optimize etmek için farklı makine öğrenimi algoritmaları deneyebiliriz.